In [ ]:
%matplotlib inline

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 512


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        if self.name == 'cn':
          for word in list(sentence):
            self.addWord(word)
        else:
          for word in sentence.split(' '):
              self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [ ]:
# ! wget http://data.statmt.org/news-commentary/v16/training/news-commentary-v16.en-zh.tsv.gz
# ! gunzip news-commentary-v16.en-zh.tsv.gz

--2021-12-18 02:45:44--  http://data.statmt.org/news-commentary/v16/training/news-commentary-v16.en-zh.tsv.gz
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://data.statmt.org/news-commentary/v16/training/news-commentary-v16.en-zh.tsv.gz [following]
--2021-12-18 02:45:44--  https://data.statmt.org/news-commentary/v16/training/news-commentary-v16.en-zh.tsv.gz
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36938999 (35M) [application/x-gzip]
Saving to: ‘news-commentary-v16.en-zh.tsv.gz’

news-commentary-v16 100%[===================>]  35.23M  14.4MB/s    in 2.4s    

2021-12-18 02:45:47 (14.4 MB/s) - ‘news-commentary-v16.en-zh.tsv.gz’ saved [36938999/36938999]



In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('news-commentary-v16.en-zh.tsv').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[s for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('en', 'cn', True)

Reading lines...
Read 322274 sentence pairs
Trimmed to 322274 sentence pairs
Counting words...
Counted words:
cn 4726
en 184192


In [ ]:
print(random.choice(pairs))

['因此美国需求激增的快速增长加上强势美元可能有助于实现欧元区急需的（国与国之间经济）再平衡。', 'As a result, rapid demand-fueled growth in the US, together with the strong dollar, could contribute to a much-needed rebalancing of the eurozone.']


In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
    words = sentence.split(' ')
    if lang.name == 'cn':
      words = list(sentence)
    return [lang.word2index[word] for word in words]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(pairs[i])
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

In [ ]:
trainIters(encoder1, attn_decoder1, 75000, print_every=500)

In [ ]:
# 3m 18s (- 493m 15s) (500 0%) 7.4923
# 6m 29s (- 480m 48s) (1000 1%) 7.0536
# 9m 34s (- 468m 58s) (1500 2%) 6.4656
# 12m 34s (- 459m 7s) (2000 2%) 6.0801
# 15m 47s (- 457m 57s) (2500 3%) 6.6461
# 18m 42s (- 448m 56s) (3000 4%) 6.1082
# 21m 39s (- 442m 30s) (3500 4%) 6.1037
# 24m 51s (- 441m 7s) (4000 5%) 6.3101
# 27m 58s (- 438m 13s) (4500 6%) 6.0242
# 30m 56s (- 433m 17s) (5000 6%) 5.9279
# 34m 4s (- 430m 35s) (5500 7%) 6.1502
# 36m 59s (- 425m 21s) (6000 8%) 5.7882
# 40m 3s (- 422m 11s) (6500 8%) 6.0404
# 43m 17s (- 420m 31s) (7000 9%) 6.2131
# 46m 33s (- 418m 58s) (7500 10%) 6.1599
# 49m 32s (- 414m 58s) (8000 10%) 5.8316
# 52m 45s (- 412m 42s) (8500 11%) 6.0925
# 55m 44s (- 408m 45s) (9000 12%) 5.8976
# 58m 49s (- 405m 36s) (9500 12%) 5.9933
# 61m 33s (- 400m 5s) (10000 13%) 5.5543
# 64m 32s (- 396m 30s) (10500 14%) 5.8250
# 67m 33s (- 393m 3s) (11000 14%) 5.9574
# 70m 46s (- 390m 48s) (11500 15%) 6.0898
# 73m 59s (- 388m 25s) (12000 16%) 6.1450
# 77m 26s (- 387m 10s) (12500 16%) 6.3143
# 80m 41s (- 384m 48s) (13000 17%) 6.0470
# 83m 57s (- 382m 27s) (13500 18%) 6.0484
# 87m 10s (- 379m 51s) (14000 18%) 6.1230
# 90m 25s (- 377m 19s) (14500 19%) 6.1979
# 93m 41s (- 374m 45s) (15000 20%) 6.0666
# 96m 58s (- 372m 17s) (15500 20%) 6.1216
# 100m 21s (- 370m 2s) (16000 21%) 6.2585
# 103m 50s (- 368m 8s) (16500 22%) 6.2053
# 107m 19s (- 366m 8s) (17000 22%) 6.1113
# 110m 48s (- 364m 3s) (17500 23%) 6.2310
# 114m 16s (- 361m 53s) (18000 24%) 6.3253
# 117m 37s (- 359m 13s) (18500 24%) 6.0541
# 120m 55s (- 356m 24s) (19000 25%) 6.0778
# 124m 20s (- 353m 54s) (19500 26%) 6.2580
# 127m 50s (- 351m 33s) (20000 26%) 6.2468
# 131m 13s (- 348m 52s) (20500 27%) 6.0498
# 134m 37s (- 346m 11s) (21000 28%) 6.2316
# 138m 2s (- 343m 30s) (21500 28%) 6.1163
# 141m 24s (- 340m 39s) (22000 29%) 6.1240
# 144m 44s (- 337m 42s) (22500 30%) 6.0711
# 148m 9s (- 334m 57s) (23000 30%) 6.1579
# 151m 36s (- 332m 15s) (23500 31%) 6.0588
# 154m 57s (- 329m 17s) (24000 32%) 6.1983
# 158m 28s (- 326m 40s) (24500 32%) 6.1283
# 161m 51s (- 323m 42s) (25000 33%) 6.0781
# 165m 27s (- 321m 11s) (25500 34%) 6.2563
# 169m 3s (- 318m 37s) (26000 34%) 6.1817
# 172m 44s (- 316m 8s) (26500 35%) 6.2616
# 176m 17s (- 313m 24s) (27000 36%) 6.1437
# 179m 44s (- 310m 27s) (27500 36%) 6.0741

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

def bleu_eval(encoder, decoder, iter, len):
  curr_pairs = [pairs[i] for i in range(iter,iter+len)]
  src = [c[0] for c in curr_pairs]
  tgt = [[c[1].split()] for c in curr_pairs]
  output = [evaluate(encoder, decoder, i)[0][:-1] for i in src]

  print("tgt is", tgt)
  print("output is", output)
  bleu_score = corpus_bleu(tgt, output, weights=(0.25, 0.25, 0.25, 0.25))

  print("-------------------------------------------------------------")
  print("Input.     :", src[0])
  print("True output:", tgt[0][0])
  print("Output     :", output[0])
  print("BLEU Score = ", bleu_score)
  print("-------------------------------------------------------------")

In [ ]:
bleu_eval(encoder1, attn_decoder1, 75000, 1000)

In [ ]:
for i in range(75000, 75000 + 10):
  pair = pairs[i]
  print("src is", pair[0])
  print("tgt is", pair[1])
  print("output is", ' '.join(evaluate(encoder1, attn_decoder1, pair[0])[0][:-1]) + '.')
  print("_______________________")

src is 特朗普的竞选以好战的民族主义和移民提案为特色 — — 从大规模修墙阻挡移民到禁止所有穆斯林进入美国 — — 以及不分青红皂白地批评美国与盟国和敌国之间的关系。
tgt is Trump’s campaign has been marked by bellicose nationalism and anti-immigrant proposals – from the construction of a massive wall to keep out immigrants to a ban on all Muslims from entering the US – as well as reckless criticisms of America’s relations with friends and foes alike.
output is The – and and and and and and and and of and and and and and and and and of.
_______________________
src is 但他却仍有可能在选举中获胜；事实上，他能够指挥数量庞大且往往非常忠实的追随者。
tgt is Yet he remains electorally viable; indeed, he commands a large and often virulently loyal following.
output is But he is that a to a that to a to a that he is to.
_______________________
src is 其中反映并且强化的国内政治紧张凸显了今天的世界秩序是多么脆弱。
tgt is The internal political strain that this reflects and reinforces highlights just how vulnerable the world order is today.
output is The that is to a of and and and.
_______________________
src is 因为风格暴躁鲁莽，特朗普似乎缺乏当今世界稳定执政的所需的知识、